In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

In [2]:
tensorflow.version.VERSION

'2.4.1'

In [3]:
# Read from: https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.names
names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', names=names)

In [4]:
# inspect
df.head(5)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
# Step 0: Reformat data
data = df.values
X = data[:, 0:4]
y = data[:, 4]

In [6]:
type(X)

numpy.ndarray

In [7]:
# Label encoding
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

Note that each of the original labels has been replaced with an integer.

In [8]:
# One-hot encoding
y_categorical = to_categorical(encoded_y)
y_categorical[:5], y[:5]

(array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]], dtype=float32),
 array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
        'Iris-setosa'], dtype=object))

In [9]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3.0, 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5.0, 3.6, 1.4, 0.2]], dtype=object)

In [10]:
std_scaler = StandardScaler().fit(X)
X_scaled = std_scaler.transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical)

In [12]:
model = Sequential(name='Iris_Model')
model.add(Dense(units=6, activation='relu', input_dim=4))
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [13]:
model.summary()

Model: "Iris_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 21        
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

model.fit(
    X_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=1
)

Epoch 1/100
4/4 [==============================] - 0s 1ms/step - loss: 0.9139 - accuracy: 0.4515
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9066 - accuracy: 0.6973
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8946 - accuracy: 0.6192
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8840 - accuracy: 0.6836
Epoch 5/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8874 - accuracy: 0.6680
Epoch 6/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8731 - accuracy: 0.6226
Epoch 7/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8709 - accuracy: 0.6674
Epoch 8/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8467 - accuracy: 0.6351
Epoch 9/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8387 - accuracy: 0.6549
Epoch 10/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8119 - accuracy: 0.6601
Epoch 11/100
4/4 [===========

In [15]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
('loss: {}, accuracy: {}'.format)(loss, accuracy)

2/2 - 0s - loss: 0.2994 - accuracy: 0.9737


'loss: 0.2993583679199219, accuracy: 0.9736841917037964'

In [16]:
def predict_class(provided_data, model_used):
    data = [list(provided_data.values())[0:4]]
    data = np.asarray(data, dtype=np.float32)
    
    scaled_data = std_scaler.transform(data)
    predictions = model_used.predict_classes(scaled_data)
    decoded_predictions = label_encoder.inverse_transform(predictions)
    
    return(scaled_data, predictions)

In [17]:
sample_data = {
    'sepal_length': 4.2,
    'sepal_width': 2.9,
    'petal_length': 1.9,
    'petal_width': 0.172,
}

In [18]:
data, predictions = predict_class(sample_data, model)

/Users/andrew/anaconda3/envs/final-project/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [19]:
data, predictions

(array([[-1.9911922 , -0.35636035, -1.0569439 , -1.3497891 ]],
       dtype=float32),
 array([0]))

In [20]:
label_encoder.inverse_transform(predictions)

array(['Iris-setosa'], dtype=object)

In [21]:
model.save('iris_model')

INFO:tensorflow:Assets written to: iris_model/assets


In [22]:
new_model = load_model('iris_model')

In [23]:
new_model.summary()

Model: "Iris_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 21        
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


In [24]:
new_model.predict_classes([[-1.9911922 , -0.35636035, -1.0569439 , -1.3497891 ]])

/Users/andrew/anaconda3/envs/final-project/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([0])

In [35]:
classes = {
    0:'Iris-setosa',
    1:'Iris-versicolor',
    2:'Iris-virginica',
}

In [36]:
classes

{0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}